# 03 · Modelagem e Avaliação

**Objetivo:** Implementar pipeline completo de treinamento e avaliação de modelos para detecção de transações suspeitas de lavagem de dinheiro.

Este notebook executa o pipeline técnico de modelagem, incluindo:
- Carregamento e pré-processamento de features
- Validação temporal para evitar data leakage
- Otimização de hiperparâmetros com Optuna + ASHA pruning
- Calibração de probabilidades
- Comparação objetiva com benchmark Multi-GNN
- Salvamento de artefatos para notebooks downstream

### Configuração Técnica
- **Dados**: Features processadas do notebook 02
- **Modelos**: XGBoost, LightGBM, RandomForest
- **Validação**: Cross-validation temporal (5 folds)
- **Otimização**: Optuna com ASHA pruning para eficiência
- **Métricas**: ROC-AUC, PR-AUC, Precision@k, Recall@FPR
- **Calibração**: Isotonic regression para probabilidades confiáveis

### Pipeline de Execução
1. Setup e configuração centralizada
2. Carregamento e pré-processamento
3. Validação temporal baseline
4. Otimização ASHA para XGBoost e LightGBM
5. Calibração e avaliação final
6. Comparação com benchmark
7. Salvamento de artefatos

### Artefatos Gerados
- Modelos otimizados e calibrados (`.pkl`)
- Resultados de otimização ASHA (`.json`)
- Métricas de calibração (`.json`)
- Comparação com benchmark (`.json`)
- Importância de features (`.json`)

Para análises executivas e apresentações, consulte o notebook `06_Executive_Summary.ipynb`.

## ▸ Configuração Centralizada

Centralizo todas as configurações do projeto para facilitar manutenção e reprodutibilidade.

In [11]:
# CONFIGURAÇÃO CENTRALIZADA
import logging
import sys
import os
from pathlib import Path
from datetime import datetime

# Configuração de logging estruturado
def setup_structured_logging(log_level=logging.INFO, log_file=None):
    """Configura logging estruturado com timestamps e níveis apropriados."""
    if log_file is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_file = f"../logs/notebook_03_{timestamp}.log"

    # Criar diretório de logs se não existir
    Path(log_file).parent.mkdir(parents=True, exist_ok=True)

    # Configuração do logger
    logging.basicConfig(
        level=log_level,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler(sys.stdout)
        ]
    )

    logger = logging.getLogger(__name__)
    logger.info("Logging estruturado configurado")
    return logger

# Dicionário de configuração centralizada
CONFIG = {
    # === CONFIGURAÇÃO GERAL ===
    'project_name': 'AML_Detection_Pipeline',
    'version': '1.0.0',
    'author': 'AML Team',
    'description': 'Pipeline de detecção de lavagem de dinheiro com ML',

    # === MODOS DE EXECUÇÃO ===
    'execution_mode': {
        'development': True,  # True para desenvolvimento, False para produção
        'quick_mode': False,  # True para testes rápidos com subamostragem
        'debug_mode': False,  # True para logs detalhados
    },

    # === CAMINHOS DE ARQUIVOS ===
    'paths': {
        'project_root': Path('..').resolve(),
        'data_dir': Path('..') / 'data' / 'processed',
        'artifacts_dir': Path('..') / 'artifacts',
        'logs_dir': Path('..') / 'logs',
        'models_dir': Path('..') / 'models',
        'features_file': 'features_with_patterns.pkl',
        'benchmark_metrics': 'gnn_benchmark_metrics.json',
        'production_config': 'production_config.json',
        'monitoring_config': 'monitoring_config.json',
    },

    # === PARÂMETROS DE DADOS ===
    'data': {
        'random_seed': 42,
        'quick_sample_size': 50000,  # Tamanho da amostra para modo rápido
        'temporal_splits': 5,  # Número de folds para validação temporal
        'test_size_ratio': 0.2,  # Proporção de teste em cada fold
    },

    # === CONFIGURAÇÃO DE MODELOS ===
    'models': {
        'xgboost': {
            'model_type': 'xgb',
            'params': {
                'n_estimators': 1000,
                'max_depth': 5,
                'learning_rate': 0.1,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'random_state': 42,
                'eval_metric': 'auc',
                'use_label_encoder': False,
                'verbosity': 0,
                'n_jobs': -1,
                'tree_method': 'hist'
            }
        },
        'lightgbm': {
            'model_type': 'lgb',
            'params': {
                'n_estimators': 1000,
                'max_depth': 6,
                'learning_rate': 0.1,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'random_state': 42,
                'verbosity': -1,
                'metric': 'auc',
                'n_jobs': 1,
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'is_unbalance': True,
                'min_child_samples': 20,
                'min_child_weight': 1e-3,
                'reg_alpha': 0.0,
                'reg_lambda': 1.0,
                'num_leaves': 31,
                'bagging_freq': 1,
                'bagging_fraction': 0.8,
                'feature_fraction': 0.8
            }
        },
        'random_forest': {
            'model_type': 'rf',
            'params': {
                'n_estimators': 80,
                'max_depth': 10,
                'min_samples_split': 10,
                'min_samples_leaf': 5,
                'random_state': 42,
                'class_weight': 'balanced',
                'n_jobs': -1
            }
        }
    },

    # === HIPERPARÂMETROS DE OTIMIZAÇÃO ===
    'optimization': {
        'optuna_trials': 50,  # Número de trials do Optuna (modo dev) ou 1 (produção)
        'early_stopping': {
            'enabled': True,
            'rounds': 20,
            'metric': 'auc',
            'min_delta': 0.001,
            'max_rounds': 1000
        },
        'asha_pruning': True,  # Usar ASHA para pruning
    },

    # === MÉTRICAS E THRESHOLDS ===
    'metrics': {
        'primary_metrics': ['roc_auc', 'average_precision'],
        'secondary_metrics': ['recall', 'precision', 'f1'],
        'aml_thresholds': [0.1, 0.3, 0.5, 0.7, 0.9],
        'business_metrics': {
            'cost_benefit_ratio': {'fp_cost': 1, 'fn_cost': 100},
            'regulatory_requirements': {
                'min_recall': 0.8,
                'max_false_positive_rate': 0.05
            }
        }
    },

    # === CALIBRAÇÃO ===
    'calibration': {
        'method': 'isotonic',  # 'isotonic' ou 'sigmoid'
        'cv_folds': 5,
        'evaluation_bins': 10,  # Para ECE
    },

    # === MONITORAMENTO E PRODUÇÃO ===
    'production': {
        'model_version': '1.0.0',
        'retraining_frequency': 'weekly',  # daily, weekly, monthly
        'drift_threshold': 0.1,
        'performance_drop_threshold': 0.05,
        'alert_channels': ['email', 'slack'],  # Canais de alerta
    },

    # === LOGGING ===
    'logging': {
        'level': 'INFO',  # DEBUG, INFO, WARNING, ERROR
        'save_logs': True,
        'log_performance_metrics': True,
        'log_model_artifacts': True,
    }
}

# Aplicar configurações baseadas no modo
if CONFIG['execution_mode']['development']:
    CONFIG['optimization']['optuna_trials'] = 5  # Menos trials em desenvolvimento
    CONFIG['logging']['level'] = 'DEBUG' if CONFIG['execution_mode']['debug_mode'] else 'INFO'
else:
    CONFIG['optimization']['optuna_trials'] = 1  # Trial único em produção
    CONFIG['logging']['level'] = 'WARNING'

# Configurar logging
logger = setup_structured_logging(
    log_level=getattr(logging, CONFIG['logging']['level']),
    log_file=CONFIG['paths']['logs_dir'] / f"notebook_03_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    if CONFIG['logging']['save_logs'] else None
)

# Log da configuração inicial
logger.info(f"Configuração centralizada carregada - Modo: {'Desenvolvimento' if CONFIG['execution_mode']['development'] else 'Produção'}")
logger.info(f"Versão do projeto: {CONFIG['version']}")
logger.info(f"Modo rápido: {CONFIG['execution_mode']['quick_mode']}")

print(" Configuração centralizada implementada")
print(f" Projeto: {CONFIG['project_name']} v{CONFIG['version']}")
print(f" Modo: {'Desenvolvimento' if CONFIG['execution_mode']['development'] else 'Produção'}")
print(f" Modo rápido: {CONFIG['execution_mode']['quick_mode']}")
print(f" Logs salvos: {CONFIG['logging']['save_logs']}")

2025-10-21 08:55:58,887 - __main__ - INFO - Logging estruturado configurado
2025-10-21 08:55:58,887 - __main__ - INFO - Configuração centralizada carregada - Modo: Desenvolvimento
2025-10-21 08:55:58,893 - __main__ - INFO - Versão do projeto: 1.0.0
2025-10-21 08:55:58,895 - __main__ - INFO - Modo rápido: False
 Configuração centralizada implementada
 Projeto: AML_Detection_Pipeline v1.0.0
2025-10-21 08:55:58,887 - __main__ - INFO - Configuração centralizada carregada - Modo: Desenvolvimento
2025-10-21 08:55:58,893 - __main__ - INFO - Versão do projeto: 1.0.0
2025-10-21 08:55:58,895 - __main__ - INFO - Modo rápido: False
 Configuração centralizada implementada
 Projeto: AML_Detection_Pipeline v1.0.0
 Modo: Desenvolvimento
 Modo rápido: False
 Logs salvos: True
 Modo: Desenvolvimento
 Modo rápido: False
 Logs salvos: True


In [12]:
# CONTROLE DE EXECUÇÃO CONDICIONAL (baseado em CONFIG)
import sys

logger.info("Verificando modo de execução...")

# Modos de execução baseados em CONFIG
EXECUTION_MODE = {
    'development': CONFIG['execution_mode']['development'],
    'quick_mode': CONFIG['execution_mode']['quick_mode'],
    'debug_mode': CONFIG['execution_mode']['debug_mode']
}

# Configurações baseadas no modo
if EXECUTION_MODE['development']:
    OPTUNA_TRIALS = CONFIG['optimization']['optuna_trials']
    CV_FOLDS = 3  # Menos folds em desenvolvimento
    logger.info("Modo desenvolvimento: trials Optuna reduzidos, validação rápida")
else:
    OPTUNA_TRIALS = 1  # Trial único em produção
    CV_FOLDS = CONFIG['data']['temporal_splits']
    logger.info("Modo produção: otimização completa")

if EXECUTION_MODE['quick_mode']:
    SAMPLE_SIZE = CONFIG['data']['quick_sample_size']
    logger.info(f"Modo rápido ativado: {SAMPLE_SIZE:,} amostras")
else:
    SAMPLE_SIZE = None
    logger.info("Modo completo: todas as amostras")

if EXECUTION_MODE['debug_mode']:
    logging.getLogger().setLevel(logging.DEBUG)
    logger.info("Modo debug ativado: logging detalhado")
else:
    logger.info("Modo normal: logging informativo")

# Função para controle condicional
def should_execute_section(section_name, force=False):
    """
    Decide se uma seção deve ser executada baseado no modo.

    Args:
        section_name: Nome da seção
        force: Forçar execução independente do modo

    Returns:
        bool: True se deve executar
    """
    if force:
        return True

    # Em modo desenvolvimento, executar apenas seções essenciais
    if EXECUTION_MODE['development']:
        essential_sections = ['setup', 'data_loading', 'preprocessing', 'validation', 'baseline']
        return section_name in essential_sections

    # Em modo produção, executar tudo
    return True

logger.info("Controle de execução configurado")
print(" Controle de execução condicional implementado")
print(f" Modo: {'Desenvolvimento' if EXECUTION_MODE['development'] else 'Produção'}")
print(f" Modo rápido: {EXECUTION_MODE['quick_mode']}")
print(f" Modo debug: {EXECUTION_MODE['debug_mode']}")

2025-10-21 08:55:58,928 - __main__ - INFO - Verificando modo de execução...
2025-10-21 08:55:58,931 - __main__ - INFO - Modo desenvolvimento: trials Optuna reduzidos, validação rápida
2025-10-21 08:55:58,931 - __main__ - INFO - Modo desenvolvimento: trials Optuna reduzidos, validação rápida
2025-10-21 08:55:58,933 - __main__ - INFO - Modo completo: todas as amostras
2025-10-21 08:55:58,937 - __main__ - INFO - Modo normal: logging informativo
2025-10-21 08:55:58,940 - __main__ - INFO - Controle de execução configurado
2025-10-21 08:55:58,933 - __main__ - INFO - Modo completo: todas as amostras
2025-10-21 08:55:58,937 - __main__ - INFO - Modo normal: logging informativo
2025-10-21 08:55:58,940 - __main__ - INFO - Controle de execução configurado
 Controle de execução condicional implementado
 Modo: Desenvolvimento
 Modo rápido: False
 Modo debug: False
 Controle de execução condicional implementado
 Modo: Desenvolvimento
 Modo rápido: False
 Modo debug: False


## ▸ Carregamento dos Dados

Carrego as features processadas do notebook anterior, com opção de modo rápido integrado.

In [14]:
# CARREGAMENTO DOS DADOS (usando CONFIG centralizado)
import joblib
import pandas as pd
import numpy as np

# Configurar modo rápido baseado em CONFIG
RUN_QUICK = CONFIG['execution_mode']['quick_mode']

# Caminhos usando CONFIG
data_dir = CONFIG['paths']['data_dir']
features_pkl = data_dir / CONFIG['paths']['features_file']

logger.info(f"Carregando dados de: {features_pkl}")
logger.info(f"Modo rápido: {RUN_QUICK}")

# Carregar dados processados
try:
    df = pd.read_pickle(features_pkl)
    # Separar features e target
    y = df['is_fraud']
    X = df.drop('is_fraud', axis=1)

    # Selecionar apenas colunas numéricas
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    X = X[numeric_cols]

    # Modo rápido (opcional)
    if RUN_QUICK:
        sample_size = CONFIG['data']['quick_sample_size']
        indices = np.random.choice(len(X), sample_size, replace=False)
        X = X.iloc[indices].reset_index(drop=True)
        y = y.iloc[indices].reset_index(drop=True)
        logger.info(f"Amostra reduzida para {sample_size:,} registros")

except Exception as e:
    logger.error(f"Erro ao carregar dados: {e}")
    raise

# Verificação visual e logging
logger.info(f"Dataset carregado: {len(X):,} transações × {X.shape[1]} features")
logger.info(f"Taxa de fraude: {y.mean():.3%}")

print(f" Dados carregados: {len(X):,} transações")
print(f" Features: {X.shape[1]} | Fraude: {y.mean():.3%}")
X.head(), y.value_counts()

 Dados carregados: 5,078,336 transações
 Features: 51 | Fraude: 0.102%

 Features: 51 | Fraude: 0.102%


(      from_bank  to_bank  amount_received    amount  Bank ID  Bank ID_to  \
 3437         70       10          1064.04   1064.04       70          10   
 3878         70     1047         33647.60  33647.60       70        1047   
 4118         70     1292         14777.01  14777.01       70        1292   
 5612         70    11471          6117.78   6117.78       70       11471   
 6266         70    11107         16561.46  16561.46       70       11107   
 
       hour_x  hour_y  source_amount_sum_7d  source_amount_mean_7d  ...  \
 3437       0       0           54015486.75           1.385012e+06  ...   
 3878       0       0           53957155.57           1.586975e+06  ...   
 4118       0       0           53916860.66           1.739254e+06  ...   
 5612       0       0               7661.17           2.553723e+03  ...   
 6266       0       0           49486451.73           2.474323e+06  ...   
 
       from_bank_frequency  from_bank_is_rare  to_bank_frequency  \
 3437           